## get original data that was fetched from ghtorrent

 Original was fetched from ghtorrent - all repos with (check previous notebook):
     - Python as a main language, 
     - Updated in 2019, 
     - Not deleted
     - Not forked

The output of this notebook is to get all repos that has a readme file and has at least 50 watchers - the output of this notebook is saved here:

*data/processed/repositories_with-readme.csv*

In [1]:
import pandas as pd
import time
import numpy as np
import os
import os.path


from github import Github



In [3]:
data_path = '../../data/ghtorrent_python_notdeleted_repos_updated2019/repositories.csv'

    
original = pd.read_csv(data_path, na_values="\\N", sep=',')
repos = original.copy()

In [162]:
print(len(original))
print(len(original[original['watchers'] >  10]))
print(len(original[original['watchers'] >  50]))
print(len(original[original['watchers'] >  500]))

615667
54377
20929
3049


In [166]:
repos = repos[repos['watchers'] >  50]
repos= repos[['id', 'url', 'owner_id', 'name', 'description', 'language',
       'created_at',   'updated_at',  'watchers']]
repos.head()

,id,url,owner_id,name,description,language,created_at,updated_at,watchers
0,31,https://api.github.com/repos/gpjt/webgl-lessons,136,webgl-lessons,https://github.com/tparisi/webgl-lessons is no...,Python,2009-10-06 12:30:19,2019-05-26 16:32:21,626
1,63,https://api.github.com/repos/pculture/unisubs,394,unisubs,Amara,Python,2011-08-19 22:04:08,2019-05-31 04:12:13,316
4,253,https://api.github.com/repos/gruns/furl,1906,furl,URL manipulation made simple.,Python,2011-11-17 01:08:16,2019-05-30 13:58:58,1050
7,367,https://api.github.com/repos/spesmilo/electrum,2253,Electrum,Electrum; Bitcoin thin client,Python,2012-08-02 16:24:30,2019-05-31 06:09:15,3755
8,390,https://api.github.com/repos/kennethreitz/requ...,796,requests,Python HTTP Requests for Humans™,Python,2011-02-13 18:38:17,2019-05-30 22:09:37,27139


## Fetch readme files for all repos with at least 50 watchers

## PLEASE DO NOT RUN THIS CODE. 
The processed repos are already saved under *data/processed* and the readme files are saved under *data/readme_files*

In [261]:

def add_save_readme(row, github):
    folder = "../../data/readme_files"
    row['has_readme'] = False
    try:
        if row['url'] is None or len(row['url'].strip()) == 0:            
            row['readme_stat'] = 'no_url'
            return row
        repo_fullname = row['url'].split('/')[-2:]
        # debug


        if repo_fullname[0].find(".") > -1 or repo_fullname[1].find(".") > -1:
            row['readme_name'] = 'many_dots'

        f_name = '{}/{}.md'.format(folder, '.'.join(repo_fullname))
        
        if os.path.isfile(f_name):
            row['has_readme'] = True
            row['readme_file_name'] ='.'.join(row['url'].split('/')[-2:])
            with open(f_name, 'r', encoding='utf-8', errors='backslashreplace') as file:
                data = file.read()
                row['readme_words_num'] = 0
                if len(data) > 0 : 
                    data.replace('\n', ' ')
                    row['readme_words_num'] = len(data.split(' '))
            
        else:    
            current_repo = github.get_repo('/'.join(repo_fullname))
            readme = current_repo.get_readme()

            readme_content = readme.decoded_content.decode("utf-8")
            row['readme_words_num'] = len(readme_content.split())

            if row['readme_words_num']> 0 :
                with open(f_name, 'w', encoding="utf-8", errors='backslashreplace') as w:
                    w.write(readme.decoded_content.decode("utf-8") )
                row['has_readme'] = True 
                row['readme_file_name'] ='.'.join(row['url'].split('/')[-2:])

                
    except Exception as e:
        print("error: {} for repo {}".format(e , row['url']))
        row['readme_stat'] = "error: {} ".format(e)

    return row
    


In [262]:
# Github api blocks us after 5000 attepmts - so the following function is called repeatidly until all files are fetched
def retry_save_readme(row, github):
    folder = "../../data/readme_files"
    if row['has_readme'] == False and  row['readme_stat'].find('API rate limit exceeded') >-1:
        try:

            if row['url'] is None or len(row['url'].strip()) == 0:            
                row['readme_stat'] = 'no_url'
                return row
            repo_fullname = row['url'].split('/')[-2:]
            # debug


            if repo_fullname[0].find(".") > -1 or repo_fullname[1].find(".") > -1:
                row['readme_name'] = 'many_dots'

            f_name = '{}/{}.md'.format(folder, '.'.join(repo_fullname))


            if os.path.isfile(f_name):
                row['has_readme'] = True
                with open(f_name, 'r', encoding='utf-8', errors='backslashreplace') as file:
                    data = file.read()
                    row['readme_words_num'] = 0
                    if len(data) > 0 : 
                        data.replace('\n', ' ')
                        row['readme_words_num'] = len(data.split(' '))

            else:    
                current_repo = github.get_repo('/'.join(repo_fullname))
                readme = current_repo.get_readme()

                time.sleep(0.5) 
                readme_content = readme.decoded_content.decode("utf-8")
                row['readme_words_num'] = len(readme_content.split())

                if row['readme_words_num']> 0 :
                    with open(f_name, 'w', encoding="utf-8") as w:
                        w.write(readme.decoded_content.decode("utf-8") )
                    row['has_readme'] = True 
        except Exception as e:
            #print("error: {} for repo {}".format(e , row['url']))
            row['readme_stat'] = "error: {} ".format(e)


    return row
    


In [263]:
github = Github(os.getenv('PAT'))
#repos = repos.apply(add_save_readme, axis=1, args=(github, ))


In [ ]:
#repos = repos.apply(retry_save_readme, axis=1, args=(github, ))

## Removing duplicates

In [270]:
# Check for duplicate
urls_value_counts_df = (repos['url'].value_counts().to_frame())
duplicate_urls = urls_value_counts_df[urls_value_counts_df['url']>1].index.tolist()
duplicate_urls

[]

In [255]:
repos.sort_values(by=['url', 'watchers'], ascending=False, inplace=True)
repos.drop_duplicates('url', keep='first', inplace=True)

## Number of fetched readme after removing duplicates

In [264]:
repos['has_readme'].value_counts()


True     20590
False      328
Name: has_readme, dtype: int64

## Type of errors for failed readme files

In [268]:
repos[repos['has_readme']==False]['readme_stat'].value_counts().to_frame()


,readme_stat
"error: 404 {""message"": ""Not Found"", ""documentation_url"": ""https://developer.github.com/v3/repos/contents/#get-the-readme""}",183
"error: 404 {""message"": ""Not Found"", ""documentation_url"": ""https://developer.github.com/v3/repos/#get""}",106
"error: 404 {""message"": ""Not Found"", ""documentation_url"": ""https://developer.github.com/v3""}",9
"error: 403 {""message"": ""Repository access blocked"", ""block"": {""reason"": ""size"", ""created_at"": ""2019-12-28T11:54:30Z"", ""html_url"": ""https://github.com/tos""}}",1
"error: 451 {""message"": ""Repository access blocked"", ""block"": {""reason"": ""dmca"", ""created_at"": ""2020-01-11T01:15:48Z"", ""html_url"": ""https://github.com/github/dmca/blob/master/2020/01/2020-01-09-alliance4creativity.md""}}",1
"error: 451 {""message"": ""Repository access blocked"", ""block"": {""reason"": ""dmca"", ""created_at"": ""2019-01-29T18:36:17Z"", ""html_url"": ""https://github.com/github/dmca/blob/master/2019/01/2019-01-28-Reuters.md""}}",1
error: unsupported encoding: None,1


## Saving the repo info that has readme files to *'data/processed/repositories_with-readme.csv'* 

In [258]:
repos_with_readme = repos[repos['has_readme'] == True]

In [260]:
#repos_with_readme.to_csv('../../data/processed/repositories_with-readme.csv', encoding="utf-8")